In [2]:
# Import packages & functions:
import pandas
import numpy

In [3]:
import pandas as pd # For working with DataFrames
import ast # For working with strings
import numpy as np # For numerical things
import re # For cleaning webtext

# For loading functions from files in data_tools directory:
import sys; sys.path.insert(0, "../../data_tools")

In [4]:
# For displaying basic DF info, storing DFs for memory efficiency, and loading a filtered DF:
from df_tools import check_df, convert_df, load_filtered_df, replace_df_nulls

# For quickly loading & saving pickle files in Python:
from quickpickle import quickpickle_dump, quickpickle_load 

# For saving and loading text lists to/from file:
from textlist_file import write_list, load_list 

# For calculating densities, years opened and closed, and school closure rates:
from df_calc import density_calc, closerate_calc

In [5]:
# Input files:
charters_path = "../../nowdata/charters_2015.pkl"
pubschools_path = "../../nowdata/pubschools_2015.pkl"

charters_small_loc = "../../nowdata/backups/charters_parsed_03-04_no-text_SMALL.csv"
ACSsmall_loc = "../data/ACS_2016_sd-merged_SMALL.csv"
ACSfull_loc = "../data/ACS_2016_sd-merged_FULL.csv"
SAIPE_loc = "../data/SAIPE_2016_sd.txt"

In [6]:
# Define variables to keep from full charter data set
keepvars = ['LEVEL', 'MEMBER', 'SE_T002_002', 'TOTETH', 'WH', 'TOTFRL', 'PCT_SE_T113_002', 
            'ESS_VALID_RATIO', 'PROG_VALID_RATIO', 'INQUIRY_RATIO', 'DISCIPLINE_RATIO', 'LSTATE', 'CMO_NAME', 
            'LEAID', 'GEO_LEAID', 'NCESSCH', 'SY_STATUS15', 'TITLEI', 'FTE', 'SCHNAM15',
            'LAT1516', 'LON1516', 'ADDRESS16', 'ALL_RLA00PCTPROF_1516', 'ALL_MTH00PCTPROF_1516']

statusyears_list = ['STATUS98', 'STATUS99', 'STATUS00', 'STATUS01', 'STATUS02', 'STATUS03', 
                    'STATUS04', 'STATUS05', 'STATUS06', 'STATUS07', 'STATUS08', 'STATUS09', 
                    'STATUS10', 'STATUS11', 'STATUS12', 'STATUS13', 'SY_STATUS', 'SY_STATUS15', 'SY_STATUS16']

In [7]:
# Load and filter charter data set
charterdf = load_filtered_df(charters_path, keepvars+statusyears_list)

# rows and cols:  (10965, 43)
# duplicates by NCESSCH: 0

Columns and # missing cases (if any): 
LEVEL: 3577 missing
MEMBER: 4034 missing
SE_T002_002: 7951 missing
TOTETH: 4034 missing
WH: 4034 missing
TOTFRL: 4034 missing
PCT_SE_T113_002: 7892 missing
ESS_VALID_RATIO: 4103 missing
PROG_VALID_RATIO: 4103 missing
INQUIRY_RATIO: 4103 missing
DISCIPLINE_RATIO: 4103 missing
LSTATE: 3577 missing
CMO_NAME: 8421 missing
LEAID: 4016 missing
GEO_LEAID: 3612 missing
NCESSCH
SY_STATUS15: 3619 missing
TITLEI: 4597 missing
FTE: 4192 missing
SCHNAM15: 3619 missing
LAT1516: 3613 missing
LON1516: 3613 missing
ADDRESS16: 3532 missing
ALL_RLA00PCTPROF_1516: 4663 missing
ALL_MTH00PCTPROF_1516: 4665 missing
STATUS98: 9600 missing
STATUS99: 9048 missing
STATUS00: 8570 missing
STATUS01: 8186 missing
STATUS02: 7749 missing
STATUS03: 7355 missing
STATUS04: 6781 missing
STATUS05: 6393 missing
STATUS06: 6178 missing
STATUS07: 6019 missing
STATUS08: 5652 missing
STATUS09: 5396 missing
STATUS10: 5017 missing
STAT

In [8]:
# Filter to only open schools
filtered_STATUS = (charterdf['SY_STATUS15'] == 1) | (charterdf['SY_STATUS15'] == 3) | (charterdf['SY_STATUS15'] == 4) | (charterdf['SY_STATUS15'] == 5) | (charterdf['SY_STATUS15'] == 8)
df = charterdf.loc[filtered_STATUS]

check_df(df, "NCESSCH")

# rows and cols:  (6947, 43)
# duplicates by NCESSCH: 0

Columns and # missing cases (if any): 
LEVEL
MEMBER: 19 missing
SE_T002_002: 4076 missing
TOTETH: 19 missing
WH: 19 missing
TOTFRL: 19 missing
PCT_SE_T113_002: 4017 missing
ESS_VALID_RATIO: 647 missing
PROG_VALID_RATIO: 647 missing
INQUIRY_RATIO: 647 missing
DISCIPLINE_RATIO: 647 missing
LSTATE
CMO_NAME: 4582 missing
LEAID: 3 missing
GEO_LEAID
NCESSCH
SY_STATUS15
TITLEI: 908 missing
FTE: 541 missing
SCHNAM15
LAT1516
LON1516
ADDRESS16: 33 missing
ALL_RLA00PCTPROF_1516: 649 missing
ALL_MTH00PCTPROF_1516: 651 missing
STATUS98: 6018 missing
STATUS99: 5762 missing
STATUS00: 5521 missing
STATUS01: 5269 missing
STATUS02: 4975 missing
STATUS03: 4703 missing
STATUS04: 4350 missing
STATUS05: 4069 missing
STATUS06: 3790 missing
STATUS07: 3530 missing
STATUS08: 3199 missing
STATUS09: 2907 missing
STATUS10: 2465 missing
STATUS11: 1934 missing
STATUS12: 1498 missing
STATUS13: 964 missing
SY_STATUS
SY_STATUS16: 33 missing


## My version (shorter)

In [33]:
def openclose_calc(statusdf, statusvars_list, uniqueid):
    """Determines year that each school was opened and (if applicable) closed.
    
    Args:
        DataFrame holding years opened and closed raw data,
        list of status-years to keep from DF,
        unique identifier for each entity (e.g., NCES school #).
        
    Returns:
        1-column DF holding year opened for each entity,
        1-column DF holding year closed for each entity."""
    
    # Trim input DFs to only relevant variables for finding close rates and merging
    statusdf = statusdf[statusvars_list]
    
    # Get length of input DF
    length = statusdf.shape[0]
    
    cols = statusvars_list # Simplify naming for more interpretable functions
    
    # Define dictionary of status-year vars as (status col : corresponding year)
    yeardict = {'STATUS98' : 1998, 'STATUS99' : 1999, 'STATUS00' : 2000, 'STATUS01' : 2001, 
                'STATUS02' : 2002, 'STATUS03' : 2003, 'STATUS04' : 2004, 'STATUS05' : 2005,
                'STATUS06' : 2006, 'STATUS07' : 2007, 'STATUS08' : 2008, 'STATUS09' : 2009,
                'STATUS10' : 2010, 'STATUS11' : 2011, 'STATUS12' : 2012, 'STATUS13' : 2013,
                'SY_STATUS' : 2014, 'SY_STATUS15' : 2015, 'SY_STATUS16' : 2016}

    # Define helper functions for workhorse algorithm below
    def checkOpenData(statusArr, currYear):
        """Check for any open status data all the years before given year. 
        If no previous data on open status, open_year should be this year.
        If there is a number with meaningful data on openness (this excludes -1, 2, 6, 7), 
        then return False; else return True."""
        
        allNeg1 = True
        for prevStat in statusArr[:cols.index(currYear)]:
            if prevStat != -1 and prevStat != 2 and prevStat != 6 and prevStat != 7:
                allNeg1 = False
        return allNeg1

    def checkClosedAfter(statusArr, currYear):
        """Check for any closure status data all the years AFTER given year.
        If no later data on closure status, closed_year should be this year.
        If there is a 2 or 6, return True; else return False."""
        
        is2or6 = False
        for afterStat in statusArr[cols.index(currYear):]:
            if afterStat == 2 or afterStat == 6:
                is2or6 = True
        return is2or6

    def checkCloseData(statusArr):
        """Check for any closure status data ALL years for a given school.
        If there exist a 2 or 6 at any point, return True; else return False."""
        
        is2or6 = False
        for stat in statusArr:
            if stat == 2 or stat == 6:
                is2or6 = True
        return is2or6

    def getyears_openclose(daterow):
        """Main algorithm to detect year opened and closed.
        Goes through certain interval of years, inclusive.
        
        Args:
            Row with dates of status (1 thru 8) for each year in cols (1998-2016).
            
        Returns:
            Year that entity opened, year that entity closed."""
        
        statusArr = [] # stores status values for each year

        # Initialize the two series to store year opened and closed
        YEAR_OPENED = numpy.NaN
        YEAR_CLOSED = numpy.NaN

        # Put things in statusArr
        for i in range(len(cols)):
            if numpy.isnan(daterow[cols[i]]):
                statusArr += [-1] #-1 if there is a nan in certain status
            else:
                statusArr += [int(daterow[cols[i]])]

        # Main logic         
        for col in cols:    
            stat = -1
            if(not numpy.isnan(daterow[col])):
                stat = int(daterow[col])

            # If there is a 2, then close_year should be this year
            if(stat == 2):
                YEAR_CLOSED = yeardict.get(col)

            # If 1, 3, 4, 5, or 8, run checkOpenData()
            # If no previous status, open_year should be this year
            if(stat in [1, 3, 4, 5, 8]):
                if checkOpenData(statusArr, col):
                    YEAR_OPENED = yeardict.get(col)

            # If 4, run checkOpenData()
            # If no previous status, open_year should be the year before this year        
            if(stat == 4):
                if checkOpenData(statusArr, col):
                    YEAR_OPENED = yeardict.get(cols[(cols.index(col) - 1)])

            # If 8, run checkClosedAfter()
            # If no closure status data (2 or 6) after this year, year_closed should be nothing
            if(stat == 8):
                if not checkClosedAfter(statusArr, col):
                    YEAR_CLOSED = numpy.NaN

            # If 6, run checkClosedAfter()
            # If no closure status data (2 or 6) after this year, year_closed should be this year
            if(stat == 6):
                if not checkClosedAfter(statusArr, col):
                    YEAR_CLOSED = yeardict.get(col)

        # If never 2 or 6, year_closed is numpy.NaN
        # Use checkCloseData() to check this
        if not checkCloseData(statusArr):
            YEAR_CLOSED = numpy.NaN
            
        return YEAR_OPENED, YEAR_CLOSED
    
    
    statusdf['YEAR_OPENED'], statusdf['YEAR_CLOSED'] = statusdf.apply(lambda x: getyears_openclose(x), axis=1)
    
    return statusdf['YEAR_OPENED'], statusdf['YEAR_CLOSED']

In [34]:
# Create new variable with years schools were opened and closed
df["YEAR_OPENED_FUN"], df["YEAR_CLOSED_FUN"] = openclose_calc(df, statusyears_list, 'NCESSCH')
#df["YEAR_OPENED_FUN"], df["YEAR_CLOSED_FUN"] = openclose_calc(df, statusyears_list, 'NCESSCH')
print("# empties:", df["YEAR_OPENED_FUN"].isnull().sum())

1999 nan
1998 nan
2003 nan
1998 nan
1998 nan
2005 nan
2005 nan
2003 nan
2003 nan
2007 nan
2008 nan
2014 nan
1998 nan
1998 nan
1998 nan
2002 nan
2004 nan
1998 nan
1998 nan
2002 nan
2005 nan
2008 nan
2010 nan
1998 nan
1998 nan
2004 nan
2005 nan
2009 nan
2005 nan
1998 nan
1998 nan
1999 nan
2004 nan
2004 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
2014 nan
1998 nan
2012 nan
2012 nan
1998 nan
1998 nan
1998 nan
2009 nan
1998 nan
1998 nan
2013 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
2006 nan
2013 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
2002 nan
1998 nan
1998 nan
1998 nan
2000 nan
1998 nan
1998 2016
1998 nan
1998 nan
2004 nan
1998 nan
2009 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1999 nan
2000 nan
2002 nan
1998 nan
1998 nan
1998 nan
1998 nan
2006 nan
2000 nan


2004 nan
2006 nan
2004 nan
2004 nan
2005 nan
2005 nan
2005 nan
2005 nan
2005 nan
2005 nan
2005 nan
2005 nan
2005 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2003 nan
2005 nan
2006 nan
2006 nan
2007 nan
2006 nan
2007 nan
2006 nan
2006 nan
2007 nan
2006 nan
2006 nan
2006 nan
2006 nan
2007 2016
2006 nan
2006 nan
2006 nan
2006 nan
2006 nan
2006 nan
2006 nan
2007 nan
2008 nan
2008 nan
2008 nan
2006 2016
2008 nan
2013 nan
2007 nan
2007 nan
2008 nan
2009 nan
2008 nan
2008 nan
2008 nan
2010 nan
2008 nan
2008 nan
2009 nan
2008 nan
2008 nan
2008 nan
2009 nan
2008 nan
2009 nan
2008 nan
2008 nan
2008 nan
2009 nan
2010 nan
2013 nan
2009 nan
2009 nan
2009 nan
2009 nan
2009 nan
2009 nan
2010 nan
2009 nan
2009 nan
2009 nan
2010 nan
2014 nan
2011 nan
2011 nan
2010 nan
2010 nan
2011 nan
2010 nan
2010 nan
2011 nan
2010 nan
2011 nan
2010 nan
2010 nan
2011 nan
2010 nan
2010 nan
2010 nan
2010 nan
2010 nan
2011 nan
2010 nan
2010 nan
2010 nan
2010 nan
2010 nan

2002 nan
2004 nan
2006 nan
2007 nan
2007 nan
2008 nan
2010 nan
2013 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
1998 nan
1998 nan
1998 nan
1998 nan
2000 nan
2000 nan
2001 nan
2001 nan
2003 nan
2003 nan
2002 nan
2002 nan
2006 nan
2006 nan
2006 nan
2008 nan
2008 nan
2011 nan
2011 nan
2010 nan
2011 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
1998 nan
1998 nan
2005 nan
2006 nan
2012 nan
2005 nan
2006 nan
2009 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2014 nan
2015 2014
2014 nan
1998 nan
1998 nan
2008 nan
1998 nan
2007 nan
2012 nan
1999 nan
2006 nan
1998 nan
2006 nan
2009 nan
1999 nan
1998 nan
1998 nan
2002 nan
2014 nan
2002 nan
2006 nan
2007 nan
2006 nan
2009 nan
2009 nan
2009 nan
2010 nan
2011 nan
2012 nan
2013 nan
2013 nan
2014 nan
2014 nan
2014 nan
2014 nan
2003 nan
2002 nan
2000 nan
2012 nan
2013 nan
2000 nan
2013 nan
2004 nan
2004 nan
2010 nan
2013 nan
2004 nan
2004 nan
2014 nan
2003 nan
2007 nan
2007 nan
2013 nan
2014 nan
2003 nan


2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2014 2016
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2006 nan
2005 nan
2005 nan
2005 nan
2009 nan
1998 nan
2009 nan
2004 nan
2010 nan
1998 nan
2001 nan
2008 nan
2001 nan
1998 nan
1999 nan
2000 nan
2000 nan
2001 nan
2002 nan
2002 nan
2007 nan
2009 nan
2009 nan
2000 nan
2010 nan
2011 nan
2011 nan
2011 nan
2011 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan


2010 2016
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2011 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2012 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan


2002 nan
1998 nan
1998 nan
2001 nan
2001 nan
2005 nan
2012 nan
2009 nan
2002 nan
2002 nan
2002 nan
2003 nan
2006 nan
2008 nan
2010 nan
2014 nan
2014 nan
2006 nan
1999 nan
2005 nan
2012 nan
1999 nan
1999 nan
2005 nan
2006 nan
2006 nan
2008 nan
2002 nan
2010 nan
2011 nan
2012 nan
2000 nan
2012 nan
2003 nan
2006 nan
2010 nan
1998 nan
2004 nan
2012 nan
2008 nan
2008 nan
2008 nan
2009 nan
2009 nan
2009 nan
2010 nan
2010 nan
2014 nan
2011 nan
2011 nan
2012 nan
2012 nan
2012 nan
2012 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2014 nan
2013 nan
2013 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 2016
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2012 nan
2012 nan
2012 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2014 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 nan
2013 2016
2013 2016
2013 2016
2013 n

2005 nan
2012 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 2016
1998 nan
1998 nan
1998 nan
2000 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 nan
1998 2016
1998 nan
1998 nan
1998 nan


ValueError: too many values to unpack (expected 2)

## Ji's version (longer)

In [ ]:
#make two series to store the newly created columns
length = df.shape[0]

#initialize the two columns
YEAR_OPENED = ["NA" for i in range(length)]
YEAR_CLOSED = [None for i in range(length)]

#the names of all the status col
status = df[['STATUS98', 'STATUS99', 'STATUS00', 'STATUS01', 'STATUS02', 'STATUS03',\
             'STATUS04', 'STATUS05', 'STATUS06', 'STATUS07', 'STATUS08', 'STATUS09',\
             'STATUS10', 'STATUS11', 'STATUS12', 'STATUS13', 'SY_STATUS', 'SY_STATUS15', 'SY_STATUS16']]

#dictionary of (status col : corresponding year)
cols = {'STATUS98' : '1998', 'STATUS99' : '1999', 'STATUS00' : '2000', 'STATUS01' : '2001',\
        'STATUS02' : '2002', 'STATUS03' : '2003', 'STATUS04' : '2004', 'STATUS05' : '2005',\
        'STATUS06' : '2006', 'STATUS07' : '2007', 'STATUS08' : '2008', 'STATUS09' : '2009',\
        'STATUS10' : '2010', 'STATUS11' : '2011', 'STATUS12' : '2012', 'STATUS13' : '2013',\
        'SY_STATUS' : '2014', 'SY_STATUS15' : '2015', 'SY_STATUS16' : '2016'}

#status as a list
b = ['STATUS98', 'STATUS99', 'STATUS00', 'STATUS01', 'STATUS02', 'STATUS03', 'STATUS04',\
     'STATUS05', 'STATUS06', 'STATUS07', 'STATUS08', 'STATUS09', 'STATUS10', 'STATUS11',\
     'STATUS12', 'STATUS13', 'SY_STATUS', 'SY_STATUS15', 'SY_STATUS16']

In [ ]:
#check all the years before the year we are looking. 
#if there is a number(except for -1, 5, 6, 7) appears, return False.
#else return True.

def checkPrevYears(statusArr, currYear):
    allNeg1 = True
    for prevStat in statusArr[:b.index(currYear)]:
        if prevStat != -1 and prevStat != 5 and prevStat != 6 and prevStat != 7:
            allNeg1 = False
    return allNeg1

In [ ]:
#check all the years after the year we are looking.
#if there is a 2 or 6, return True. else return False

def checkAfterYears(statusArr, currYear):
    is2or6 = False
    for afterStat in statusArr[b.index(currYear):]:
        if afterStat == 2 or afterStat == 6:
            is2or6 = True
    return is2or6

In [ ]:
def checkAfter6(statusArr, currYear):
    anythingAfter = False
    for afterStat in statusArr[b.index(currYear):]:
        if afterStat != None:
            anythingAfter = True
    return anythingAfter

In [ ]:
#check all the years for a certain school.
#if there exist a 2 or 6, return True; else return False;

def checkAllYears(statusArr):
    is2or6 = False
    for stat in statusArr:
        if stat == 2 or stat == 6:
            is2or6 = True
    return is2or6

In [ ]:
#main algorithm for a certain interval, inclusive.
#go through each row
YEAR_OPENED = ["NA" for i in range(length)]
YEAR_CLOSED = ["NA" for i in range(length)]
for index, row in status.iterrows():
    #only calculate for the input interval
    if index >= 0 and index < df.shape[0]:
        statusArr = [] #a list used to store all status for a year

        #putting things in statusArr
        for i in range(len(cols)):
            if np.isnan(row[b[i]]):
                statusArr += [-1] #-1 if there is a nan in certain status
            else:
                statusArr += [int(row[b[i]])]

        #main logic         
        for col in cols:    
            stat = -1
            if(not np.isnan(row[col])):
                stat = int(row[col])

            #if there is a 2, the close_year should be this year
            if(stat == 2):
                YEAR_CLOSED[index] = cols.get(col)

            #if 1,3 or 8, checkPreYears, if no previous status, open_year should be this year
            if(stat == 1 or stat == 3 or stat == 8):
                if checkPrevYears(statusArr, col):
                    YEAR_OPENED[index] = cols.get(col)

            #if 4, checkPrevYears. If no previous status, open_year should be the year before this year        
            if(stat == 4):
                if checkPrevYears(statusArr, col):
                    YEAR_OPENED[index] = cols.get(b[(b.index(col) - 1)])

            #if 8, checkLaterYear. If nothing after, year_closed should be this year
            if(stat == 8):
                if not checkAfterYears(statusArr, col):
                    YEAR_CLOSED[index] = 'NA'
            
            #if nothing after 6, year_closed should be this year
            if(stat == 6):
                if not checkAfter6(statusArr, col):
                    YEAR_CLOSED[index] = cols.get(col)

        #if never 2 or 6, year_closed is 'NA'
        if not checkAllYears(statusArr):
            YEAR_CLOSED[index] = 'NA'

In [ ]:
# This is used for testing only
for index in range(0, df.shape[0]):
    if YEAR_CLOSED[index] == None or YEAR_OPENED[index] == None:
        print(index,"|", YEAR_OPENED[index],"|", YEAR_CLOSED[index])

In [ ]:
# Create two dataframe for the mappings
label = ['YEAR_OPENED']
label2 = ['YEAR_CLOSED']
dfOpened = pandas.DataFrame(YEAR_OPENED, columns = label)
dfClosed = pandas.DataFrame(YEAR_CLOSED, columns = label2)

In [ ]:
# Concatenating the two mapping dataframe to our source
# Here we use concat instead of merge because the sequence of NCESSCH is preserved
result = pandas.concat([df, dfOpened, dfClosed], axis=1)

In [ ]:
result

In [ ]:
# Match new columns to order in original DF
opened = pandas.merge(df, result, how='outer', on=['NCESSCH'])['YEAR_OPENED']
closed = pandas.merge(df, result, how='outer', on=['NCESSCH'])['YEAR_CLOSED']

In [ ]:
opened

In [ ]:
df["YEAR_OPENED"], df["YEAR_CLOSED"] = opened, closed

In [ ]:
check_df(df, "NCESSCH")

In [ ]:
df